In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from glob import glob
import os
import pickle
import sys
from collections import defaultdict

# visualizations with plotly
from _plotly_future_ import v4_subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.subplots as tls
import plotly.figure_factory as ff
import plotly.io as pio
pio.templates.default = 'plotly_white'

# new default
colors = ["#636efa", "#EF553B", "#00cc96", "#ab63fa", "#FFA15A", 
          "#19d3f3", "#FF6692", "#B6E880", "#FF97FF", "#FECB52"]

In [2]:
HOME_DIR = '/mnt/c/umd/demos/FC_geodesic'
        
conditions = ['condition1']
N = 20
kROI = 300
methods = ['demo']
distance = ['pearson', 'geodesic']

In [13]:
def _load_accuracy(files):
    accuracy = []
    sem = []
    for file in files:
        with open(file, 'rb') as f:
            a = pickle.load(f)
        accuracy.append(a)
        
    results = {}
    ind = np.argmax(accuracy)    
    results['accuracy'] = accuracy[ind]
    
    return results
            
def _get_accuracy(cond, dist, trim_method, N, kROI):
    '''
    inputs -
    cond(ition): RS WM ...
    dist(ance): pearson or geodesic
    trim_method: trim or truncated or full blocks
    N: number of subjects
    kROI: number of ROI
    
    output -
    accuracy['LR'] when train data has phase LR
    accuracy['RL'] when train data has phase RL
    test data always has opposite phase
    '''
    LOAD_DIR = (HOME_DIR + 
                '/results/%s/N_%d_kROI_%d/whole_brain/accuracy' %(
               trim_method, N, kROI))
    
    regex = '*%s*.pkl' %(dist)
    files = [y for x in os.walk(LOAD_DIR) 
             for y in glob(os.path.join(x[0], regex))]
    # filter by bootstrap and by condition
    LR_files = []
    RL_files = []
    for file in files:
        if 'B_' not in file:
            LR = 'train_%s_LR' %(cond)
            RL = 'train_%s_RL' %(cond)
            if LR in file:
                LR_files.append(file)
            elif RL in file:
                RL_files.append(file)
                           
    accuracy = {}
    accuracy['LR'] = _load_accuracy(LR_files)
    accuracy['RL'] = _load_accuracy(RL_files)
    
    return accuracy

def _plot_accuracy(accuracy, trim_method):
    
    fig = go.Figure()
    for dist in accuracy:
        acc = accuracy[dist]
        x = [ii for ii in acc]
        y = []
        error = []
        for cond in acc:
            #
            a = (acc[cond]['LR']['accuracy'] +
                 acc[cond]['RL']['accuracy'])/2               
            y.append(a)
                
        fig.add_trace(go.Bar(x=x, 
                             y=y,
                             name=dist.capitalize()))
    
    title = trim_method
    fig.layout.update(title=title,
                      font=dict(family='helvetica'),
                      legend=go.layout.Legend(
                              x=0, y=-0.08,
                              font=dict(size=16,
                                        color="black")
                             ),
                      legend_orientation='h',
                      height=500,
                      width=400
                     )
    fig.update_xaxes(tickfont=dict(size=16))
    fig.update_yaxes(title=dict(text='Accuracy', font=dict(size=20)),
                     tickfont=dict(size=16),
                     range= [0.2, 1])

    py.iplot(fig)

## no bootstrap analysis

In [14]:
for trim_method in methods:    
    accuracy = defaultdict(dict)
    for cond in conditions:
        for dist in distance:
            accuracy[dist][cond] = _get_accuracy(cond, dist, trim_method,
                                                 N, kROI)
    _plot_accuracy(accuracy, trim_method)